In [2]:
import torch
import torchvision
import matplotlib.pyplot as plt
import _pickle as cPickle
import numpy as np
import pandas as pd
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as utils
from torch.autograd import Variable

In [3]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [4]:
batch1_dict = unpickle("CIFAR10/cifar-10-batches-py/data_batch_1")
batch2_dict = unpickle("CIFAR10/cifar-10-batches-py/data_batch_1")
batch3_dict = unpickle("CIFAR10/cifar-10-batches-py/data_batch_1")
batch4_dict = unpickle("CIFAR10/cifar-10-batches-py/data_batch_1")
batch5_dict = unpickle("CIFAR10/cifar-10-batches-py/data_batch_1")
test_dict = unpickle("CIFAR10/cifar-10-batches-py/data_batch_1")
meta_dict = unpickle("CIFAR10/cifar-10-batches-py/data_batch_1")

In [5]:
#Checking keys of all the dataset dictionaries

#batch1_dict.keys()
test_dict.keys()
#meta_dict.keys()

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])

In [6]:
xbatch_1 = batch1_dict[b'data']
xbatch_2 = batch2_dict[b'data']
xbatch_3 = batch3_dict[b'data']
xbatch_4 = batch4_dict[b'data']
xbatch_5 = batch5_dict[b'data']
xtest = test_dict[b'data']
ybatch_1 = batch1_dict[b'labels']
ybatch_2 = batch2_dict[b'labels']
ybatch_3 = batch3_dict[b'labels']
ybatch_4 = batch4_dict[b'labels']
ybatch_5 = batch5_dict[b'labels']
ytest = test_dict[b'labels']

In [7]:
xbatch1 = np.array(xbatch_1)
xbatch2 = np.array(xbatch_2)
xbatch3 = np.array(xbatch_3)
xbatch4 = np.array(xbatch_4)
xbatch5 = np.array(xbatch_5)
x_test = np.array(xtest)
ybatch1 = np.array(ybatch_1)
ybatch2 = np.array(ybatch_2)
ybatch3 = np.array(ybatch_3)
ybatch4 = np.array(ybatch_4)
ybatch5 = np.array(ybatch_5)
y_test = np.array(ytest)

In [8]:
x_train = np.concatenate((xbatch1, xbatch2, xbatch3, xbatch4))
y_train = np.concatenate((ybatch1, ybatch2, ybatch3, ybatch4))
x_valid = xbatch5
y_valid = ybatch5

In [9]:
np.shape(x_train)

(40000, 3072)

In [10]:
def image_reshape(dataset):
  new_reshaped = []
  for i in range (np.shape(dataset)[0]):
    r = dataset[i][:1024]
    g = dataset[i][1024:2048]
    b = dataset[i][2048:3072]
    r = np.reshape(r,(32, 32))
    g = np.reshape(g,(32, 32))
    b = np.reshape(b,(32, 32))
    rgb = np.dstack((r,g,b))
    new_reshaped.append(rgb)
    
  new_reshaped = np.array(new_reshaped)
  return new_reshaped

In [11]:
x_train = image_reshape(x_train)
x_test = image_reshape(x_test)
x_valid = image_reshape(x_valid)

In [12]:
x_train = x_train.transpose((0, 3, 1, 2))
x_test = x_test.transpose((0, 3, 1, 2))
x_valid = x_valid.transpose((0, 3, 1, 2))

In [13]:
xtrainTensor = torch.from_numpy(x_train)
xtestTensor = torch.from_numpy(x_test)
xvalidTensor = torch.from_numpy(x_valid)

ytrainTensor = torch.from_numpy(y_train).long()
ytestTensor = torch.from_numpy(y_test).long()
yvalidTensor = torch.from_numpy(y_valid).long()

In [14]:
train_dataset = utils.TensorDataset(xtrainTensor, ytrainTensor) 
test_dataset = utils.TensorDataset(xtestTensor, ytestTensor)
valid_dataset = utils.TensorDataset(xvalidTensor, yvalidTensor)

In [15]:
batch_size = 128

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)
valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [16]:
n_iters = 2048
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)
num_epochs = 6

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 7, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(7, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [19]:
model = Net()
model = model.float()

In [20]:
criterion = nn.CrossEntropyLoss()

In [21]:
learning_rate = 0.01
moment = 0.4
#optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum = moment, nesterov = False)
optimizer = torch.optim.Adamax(model.parameters(), lr = learning_rate)

In [22]:
losses = [];
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.float())
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        losses.append(loss.data[0]);
        
        if (i+1) % 100 == 0:
            print ('Epoch : %d/%d, Iter : %d/%d,  Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

C:\Users\Paritosh\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  
C:\Users\Paritosh\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch : 1/6, Iter : 100/312,  Loss: 2.3058
Epoch : 1/6, Iter : 200/312,  Loss: 2.3024
Epoch : 1/6, Iter : 300/312,  Loss: 2.3043
Epoch : 2/6, Iter : 100/312,  Loss: 2.3051
Epoch : 2/6, Iter : 200/312,  Loss: 2.3020
Epoch : 2/6, Iter : 300/312,  Loss: 2.3004
Epoch : 3/6, Iter : 100/312,  Loss: 2.3046
Epoch : 3/6, Iter : 200/312,  Loss: 2.3008
Epoch : 3/6, Iter : 300/312,  Loss: 2.2920
Epoch : 4/6, Iter : 100/312,  Loss: 2.3009
Epoch : 4/6, Iter : 200/312,  Loss: 2.2998
Epoch : 4/6, Iter : 300/312,  Loss: 2.2905
Epoch : 5/6, Iter : 100/312,  Loss: 2.2967


KeyboardInterrupt: 